# Telecom Churn Prediction

# Problem statement

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business
goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn. In this project, you will analyze customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn, and identify the main indicators of churn.

In this competition, your goal is *to build a machine learning model that is able to predict churning customers based on the features provided for their usage.*

**Customer behaviour during churn:**

Customers usually do not decide to switch to another competitor instantly, but rather over a
period of time (this is especially applicable to high-value customers). In churn prediction, we
assume that there are three phases of customer lifecycle :

1. <u>The ‘good’ phase:</u> In this phase, the customer is happy with the service and behaves as usual.

2. <u>The ‘action’ phase:</u> The customer experience starts to sore in this phase, for e.g. he/she gets a compelling offer from a competitor, faces unjust charges, becomes unhappy with service quality etc. In this phase, the customer usually shows different behaviour than the ‘good’ months. It is crucial to identify high-churn-risk customers in this phase, since some corrective actions can be taken at this point (such as matching the competitor’s offer/improving the service quality etc.)

3. <u>The ‘churn’ phase:</u> In this phase, the customer is said to have churned. In this case, since you are working over a four-month window, the first two months are the ‘good’ phase, the third month is the ‘action’ phase, while the fourth month (September) is the ‘churn’ phase.

#### Below steps will be folowed for evaluation
1. Loading dependencies & datasets

# 1. Loading dependencies & datasets

Lets start by loading our dependencies. We can keep adding any imports to this cell block, as we write mode and mode code.

In [ ]:
pip install missingno

In [ ]:
#Data Structures
import pandas as pd
import numpy as np
import re
import os

### For installing missingno library, type this command in terminal
#pip install missingno

import missingno as msno

#Sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score

#Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Class Imbalance
from imblearn.over_sampling import SMOTE

#Others
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Next, we load our datasets and the data dictionary file.

The **train.csv** file contains both dependent and independent features, while the **test.csv** contains only the independent variables. 

So, for model selection, I will create our own train/test dataset from the **train.csv** and use the model to predict the solution using the features in unseen test.csv data for submission.

### Derived attribute related functions

In [ ]:
#The above data is with respect to data recharge
# We can create new column indicating data recharge and substitute null columns with 0 value


def create_data_rechange_info(df):
    df['data_rechange_6'] = df['date_of_last_rech_data_6'].isnull()
    df['data_rechange_7'] = df['date_of_last_rech_data_7'].isnull()
    df['data_rechange_8'] = df['date_of_last_rech_data_8'].isnull()

    del df['date_of_last_rech_data_6']
    del df['date_of_last_rech_data_7']
    del df['date_of_last_rech_data_8']

    df_len = len(df)*0.7
    i=0
    for col in df.columns:
        if df[col].isnull().sum() >= df_len:
            df[col].fillna(0)
    return df

def create_last_rechange_days(df):
    df['last_recharge_days'] = (pd.to_datetime('2014-08-31') - df[['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8']].apply(pd.to_datetime).max(axis=1)).dt.days
    del df['date_of_last_rech_6']
    del df['date_of_last_rech_7']
    del df['date_of_last_rech_8']
    return df

def create_new_variables(df):
    df = create_data_rechange_info(df)
    df = create_last_rechange_days(df)
    return df

## Set display limits
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#COMMENT THIS SECTION INCASE RUNNING THIS NOTEBOOK LOCALLY

#Checking the kaggle paths for the uploaded datasets
#import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#INCASE RUNNING THIS LOCALLY, PASS THE RELATIVE PATH OF THE CSV FILES BELOW
#(e.g. if files are in same folder as notebook, simple write "train.csv" as path)

data = pd.read_csv("/kaggle/input/telecom-churn-case-study-hackathon-c53/train.csv")
unseen = pd.read_csv("/kaggle/input/telecom-churn-case-study-hackathon-c53/test.csv")
sample = pd.read_csv("/kaggle/input/telecom-churn-case-study-hackathon-c53/sample.csv")
data_dict = pd.read_csv("/kaggle/input/telecom-churn-case-study-hackathon-c53/data_dictionary.csv")

#data = pd.read_csv("train.csv")
#unseen = pd.read_csv("test.csv")
#sample = pd.read_csv("sample.csv")
#data_dict = pd.read_csv("data_dictionary.csv")


print('Train data shape - ',data.shape)
print('Test data shape - ',unseen.shape)
print('Sample data shape - ',sample.shape)
print('Data dictionary shape - ', data_dict.shape)

In [ ]:
data.describe()

### Check data type of columns

In [ ]:
data.info()
# data type are correctly assigned

### Columns with NULL value

In [ ]:
## Columns with Null values
filtered_data = data.loc[:, data.isnull().sum() != 0]
filtered_data.isnull().sum()

## 2. Data cleaning and formatting

### Delete columns with only one value

In [ ]:
df_len = len(data)*0.9
i=0
for col in data.columns:
    if len(data[col].value_counts()) == 1:
        print(data[col].value_counts())

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data[data['loc_og_t2o_mou'].isnull()]['churn_probability'].value_counts()

In [ ]:
df_len = len(data)*0.9
i=0
for col in data.columns:
    if len(data[col].value_counts()) == 1:
        i=i+1
        print(i,'.', col,'.',data[col][0],'.',round(data[col].value_counts().iloc[0]*100/len(data),2),'%')
        del data[col]

### Delete columns with more than 90% same values

In [ ]:
df_len = len(data)*0.9
i=0
l = []
for col in data.columns:
    if data[col].value_counts().iloc[0] >= df_len:
        i=i+1
        print(i,'.', col,'.',data[col].value_counts().iloc[0],'.',round(data[col].value_counts().iloc[0]*100/len(data),2),'%')
        del data[col]

### Removing columns with more than 70 % null values

In [ ]:
df_len = len(data)*0.7
i=0
print('Columns with more than 70% missing values')
for col in data.columns:
    if data[col].isnull().sum() >= df_len:
        i=i+1
        print(i,'.', col,'.',data[col].isnull().sum(),'.', round((data[col].isnull().sum()/len(data))*100,2))
        #del data[col]

#### <span style="color:blue">Observation :-</span>
Since same count of data for rechange related columns are missing, hence will not delete it.
Will handle in derived data section.
Create new attribute to indicate has data recharge or not and consider default value for NA cell as Zero.

In [ ]:
data = create_data_rechange_info(data)
unseen = create_data_rechange_info(unseen)

### Removing columns not required for analysis

In [ ]:
del data['id'] # primary key

### Removing rows with all null values in July, Aug, Sept

In [ ]:
# Get all unique prefixes from column names
prefixes = set('_'.join(col.split('_')[0:-1]) for col in data.columns if col.split('_')[-1] in ['6','7','8'])
# Create a list to store filtered DataFrames for each prefix
filtered_dfs = []
print('Shape before row delete - ', data.shape)
# Iterate over prefixes and filter rows where all columns with the same prefix are null
for prefix in prefixes:
    # Get columns with the current prefix
    cols_with_prefix = [col for col in data.columns if col.startswith(prefix)]
    
    # Filter rows where all columns with the same prefix are null
    filtered_df = data[data[cols_with_prefix].isnull().all(axis=1)]
    if len(filtered_df) > 0:
        print(prefix,'_*', ' count - ', len(filtered_df))
    # Append the filtered DataFrame to the list
    data = data[~data[cols_with_prefix].isnull().all(axis=1)]
    
print('Shape after row delete - ', data.shape)

### Check for duplicates

In [ ]:
data[data.duplicated()]

### Add default value 0 to numerical values

In [ ]:
for col in data.select_dtypes(include=['float']):
    data[col] = data[col].fillna(0)

In [ ]:
filtered_data = data.loc[:, data.isnull().sum() != 0]
filtered_data.isnull().sum()

### Derived attributes

In [ ]:
df_len = len(data)*0.7
i=0
print('Columns with more than 50% missing values deleted')
for col in data.columns:
    if data[col].isnull().sum() >= df_len:
        i=i+1
        print(i,'.', col,'.',data[col].isnull().sum(),'.', round((data[col].isnull().sum()/len(data))*100,2))

#### <span style="color:blue">Observation :-</span>
The value in above columns is only for data rechange related columns
So we can create a column to depict data recharge and fill null numerical values with 0 instead of NA

### derive last recharge in days from dates

In [ ]:
data = create_last_rechange_days(data)
unseen = create_last_rechange_days(unseen)

In [ ]:
## Columns with Null values
filtered_data = data.loc[:, data.isnull().sum() != 0]
filtered_data.isnull().sum()

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
data[data['onnet_mou_8'].isnull()]['churn_probability'].value_counts()

In [ ]:
data['churn_probability'].value_counts()

In [ ]:
## Columns with Null values
filtered_data = unseen.loc[:, unseen.isnull().sum() != 0]
filtered_data.isnull().sum()

## 3. Exploratory Data analysis 

### Univariant analysis

In [ ]:
plt.pie(data['churn_probability'].value_counts().values,labels=['Not Churn','Churn'], autopct='%.2f%%')
plt.axis('equal')
plt.title('Churn vs Not Churn')
plt.show()

> <span style="color:blue">Observation</span> - Only 10% of data has churn related information hence class imbalance handling techniques to be used

In [ ]:
# Get all unique prefixes from column names
month_columns = list(col for col in data.columns if 'total' in col)

# figure size
plt.figure(figsize=(20, 15))

# heatmap
sns.heatmap(data[month_columns].corr(), cmap="Wistia", annot=True)
plt.show()

### Drop month related columns which are highly correlated with total

In [ ]:
correlation_matrix = data.corr()

# Set the correlation threshold (adjust as needed)
correlation_threshold = 0.7
total_columns = set(col for col in data.columns if 'total' in col)
month_columns = set(col for col in data.columns if 'total' not in col and col.split('_')[-1] in ['6','7','8'])

# Find columns with high correlation
highly_correlated_columns = set()
for i in total_columns:
    for j in month_columns:
        if abs(correlation_matrix[i][j]) > correlation_threshold:
            print(j," --- ",correlation_matrix[i][j])
            highly_correlated_columns.add(j)

# 'highly_correlated_columns' now contains the names of columns with high correlation
print(highly_correlated_columns)

data.drop(columns=highly_correlated_columns, inplace=True)

### Drop month related columns which are highly correlated

correlation_matrix = data.corr()

# Set the correlation threshold (adjust as needed)
correlation_threshold = 0.8
month_columns = set(col for col in data.columns if 'total' not in col and col.split('_')[-1] in ['6','7','8'])

# Find columns with high correlation
highly_correlated_columns = set()
for i in month_columns:
    for j in month_columns:
        if abs(correlation_matrix[i][j]) > correlation_threshold and i!=j:
            highly_correlated_columns.add(j)

# 'highly_correlated_columns' now contains the names of columns with high correlation
print(highly_correlated_columns)

data.drop(columns=highly_correlated_columns, inplace=True)

correlation_matrix = data.corr()

# Set the correlation threshold (adjust as needed)
correlation_threshold = 0.80
month_columns = set(col for col in data.columns if 'total' not in col and col.split('_')[-1] in ['6','7','8'])

# Find columns with high correlation
highly_correlated_columns = set()
for i in data.columns:
    for j in data.columns:
        if abs(correlation_matrix[i][j]) > correlation_threshold and i!=j:
            print(j," --- ",correlation_matrix[i][j])
            highly_correlated_columns.add(j)

# 'highly_correlated_columns' now contains the names of columns with high correlation
print(highly_correlated_columns)

data.drop(columns=highly_correlated_columns, inplace=True)

In [ ]:
data.columns

In [ ]:
data_dict

For the purpose of this **starter notebook**, we I will restrict the dataset to only a small set of variables. 

The approach I use here is to understand each Acronym, figure our what variable might be important and filter out variable names based on the combinations of acrynoms using REGEX. So, if I want the total minutes a person has spent on outgoing calls, I need acronyms, TOTAL, OG and MOU. So corresponding regex is ```total.+og.+mou```

In [ ]:
data.head()

Let's look at each variable's datatype:

In [ ]:
data.info(verbose=1)

Let's also summarize the features using the df.describe method:

In [ ]:
data.describe(include="all")

# 2. Create X, y and then Train test split

Lets create X and y datasets and skip "circle_id" since it has only 1 unique value

In [ ]:
y = data.pop('churn_probability')
X = data

X.shape, y.shape

Splitting train and test data to avoid any contamination of the test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train.head()

# 4. Exploratory Data Analysis & Preprocessing

Lets start by analysing the univariate distributions of each feature.

In [ ]:
plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
sns.boxplot(data = X_train)

### 4.1 Handling outliers

The box plots of these features show there a lot of outliers. These can be capped with k-sigma method.

In [ ]:
def cap_outliers(array, k=3):
    upper_limit = array.mean() + k*array.std()
    lower_limit = array.mean() - k*array.std()
    array[array<lower_limit] = lower_limit
    array[array>upper_limit] = upper_limit
    return array

In [ ]:
X_train_filtered1 = X_train.apply(cap_outliers, axis=0)

plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
sns.boxplot(data = X_train_filtered1)

### 4.2 Feature scaling

Lets also scale the features by scaling them with Standard scaler (few other alternates are min-max scaling and Z-scaling).

In [ ]:
new_vars = X_train_filtered1.columns

In [ ]:
scale = StandardScaler()
X_train_filtered2 = scale.fit_transform(X_train_filtered1)

In [ ]:
plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
sns.boxplot(data = pd.DataFrame(X_train_filtered2, columns=new_vars))

You can perform feature transformations at this stage. 

1. **Positively skewed:** Common transformations of this data include square root, cube root, and log.
2. **Negatively skewed:** Common transformations include square, cube root and logarithmic.

Please read the following link to understand how to perform feature scaling and preprocessing : https://scikit-learn.org/stable/modules/preprocessing.html
 
Lets also plot the correlations for each feature for bivariate analysis.

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(pd.DataFrame(X_train_filtered2, columns=new_vars).corr())

In [ ]:
#Distribution for the churn probability
sns.histplot(y_train)

# 5. Feature engineering and selection

Let's understand feature importances for raw features as well as components to decide top features for modelling.

In [ ]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train_filtered2, y_train)

In [ ]:
feature_importances = pd.DataFrame({'col':new_vars, 'importance':rf.feature_importances_})

In [ ]:
plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
plt.bar(feature_importances['col'], feature_importances['importance'])

At this step, you can create a bunch of features based on business understanding, such as 
1. "average % gain of 3g volume from month 6 to 8" - (growth or decline of 3g usage month over month?)
2. "ratio of total outgoing amount and age of user on network" - (average daily usage of a user?)
3. "standard deviation of the total amount paid by user for all services" - (too much variability in charges?)
4. etc..

Another way of finding good features would be to project them into a lower dimensional space using PCA. PCA creates components which are a linear combination of the features. This then allows you to select components which explain the highest amount of variance.

Lets try to project the data onto 2D space and plot. **Note:** you can try TSNE, which is another dimensionality reduction approach as well. Check https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html for moree details.

In [ ]:
pca = PCA(0.95)
pca_components = pca.fit_transform(X_train_filtered2)
sns.scatterplot(x=pca_components[:,0], y=pca_components[:,1], hue=y_train)

In [ ]:
sns.scatterplot(x=pca_components[:,1], y=pca_components[:,2], hue=y_train)

In [ ]:
len(rf.feature_importances_)

In [ ]:
pca_components.shape

Let's also check which of the components have high feature importances towards the end goal of churn prediction.

In [ ]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(pca_components, y_train)

feature_importances = pd.DataFrame({'col':['component_'+str(i) for i in range(len(rf.feature_importances_))], 
                                    'importance':rf.feature_importances_})

plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
plt.bar(feature_importances['col'], feature_importances['importance'])

# 6. Model building

Let's build a quick model with logistic regression and the first 2 PCA components.

# Linear Regression

The steps of this pipeline would be the following, but this is only one type of pipeline -
1. Imputation
2. Scaling
3. PCA
4. Classification model


In [ ]:
imp = SimpleImputer(strategy='constant', fill_value=0)
scale = StandardScaler()
pca = PCA(n_components=10, random_state=42)
lr = LogisticRegression(max_iter=1000, tol=0.001,random_state=42)

In [ ]:
pipe_lr = Pipeline(steps = [('imputation',imp),
                         ('scaling',scale),
                         ('pca',pca),
                         ('model',lr)])

In [ ]:
pipe_lr.fit(X_train[new_vars], y_train)

In [ ]:
train_score = pipe_lr.score(X_train[new_vars], y_train)
print("Training accuracy:", train_score)

In [ ]:
test_score = pipe_lr.score(X_test[new_vars], y_test)
print("Test accuracy:", test_score)

Let's make a confusion matrix to analyze how each class is being predicted by the model.

In [ ]:
confusion_matrix(y_train, pipe_lr.predict(X_train[new_vars]))

In [ ]:
confusion_matrix(y_test, pipe_lr.predict(X_test[new_vars]))

We can see a high amount of type 2 error. Due to class imbalance, the model is clearly trying to predict majority of the cases as class 0. Understanding how to handle class imbalance in classification models might be the key to winning this competition :) (hint!)

In [ ]:
precision_score(y_test, pipe_lr.predict(X_test[new_vars]))

In [ ]:
recall_score(y_test, pipe_lr.predict(X_test[new_vars]))

# Random Forest

In [ ]:
from imblearn.pipeline import Pipeline as ImbPipeline
# Define your pipeline
pipe_rf = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('pca', PCA(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42))
])
# Define the parameter grid for GridSearchCV
# Define the parameter grid for GridSearchCV
param_grid = {
    'smote__sampling_strategy': [0.5, 0.75, 1.0], 
    'rf__n_estimators': [20, 30, 50],  # Adjust as needed
    'rf__max_depth': [10, 20, 30], # Adjust as needed
    # Add other parameters you want to tune
}

# Create the GridSearchCV object
grid_search = GridSearchCV(pipe_rf, param_grid, cv=5, scoring='accuracy')  # Adjust scoring and cv as needed

# Fit the model
grid_search.fit(X_train, y_train)

# Access the best parameters and best estimator
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [ ]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
rf.fit(pca_components, y_train)

feature_importances = pd.DataFrame({'col':['component_'+str(i) for i in range(len(rf.feature_importances_))], 
                                    'importance':rf.feature_importances_})

plt.figure(figsize=(15,8))
plt.xticks(rotation=45)
plt.bar(feature_importances['col'], feature_importances['importance'])

In [ ]:
pipe_rf = Pipeline(steps = [('imputation',imp),
                         ('scaling',scale),
                         ('pca',pca),
                         ('model',rf)])

In [ ]:
pipe_rf.fit(X_train[new_vars], y_train)

In [ ]:
train_score = pipe_rf.score(X_train[new_vars], y_train)
print("Training accuracy:", train_score)

In [ ]:
test_score = pipe_rf.score(X_test[new_vars], y_test)
print("Testing accuracy:", test_score)

# 7. Creating submission file

For submission, we need to make sure that the format is exactly the same as the sample.csv file. It contains 2 columns, id and churn_probability

In [ ]:
sample.head()

The submission file should contain churn_probability values that have to be predicted for the unseen data provided (test.csv)

In [ ]:
unseen.head()

Lets first select the columns that we want to work with (or create them, if you have done any feature engineering)

In [ ]:
new_vars

In [ ]:
new_vars = [col for col in new_vars if col != 'churn']

In [ ]:
submission_data = unseen.set_index('id')[new_vars]
submission_data.shape

Next, lets create a new column in the unseen dataset called churn_probability and use the model pipeline to predict the probabilities for this data

In [ ]:
unseen['churn_probability'] = pipe_rf.predict(submission_data)
output = unseen[['id','churn_probability']]
output.head()

Finally, lets create a csv file out of this dataset, ensuring to set index=False to avoid an addition column in the csv.

In [ ]:
output.to_csv('submission.csv',index=False)

You can now take this file and upload it as a submission on Kaggle.

In [ ]:
output['churn_probability'].value_counts()